In [1]:
from utils import tokenize, load_curpus
import numpy as np

#### 加载数据

In [2]:
import pandas as pd
train_data = load_curpus("weibo2018/train.txt")
test_data = load_curpus("weibo2018/test.txt")
train_df = pd.DataFrame(train_data, columns=["content", "sentiment"])
test_df = pd.DataFrame(test_data, columns=["content", "sentiment"])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.498 seconds.
Prefix dict has been built succesfully.


In [3]:
train_df.head()

,content,sentiment
0,"[书中, 自有, 黄金屋, 书中, 自有, 颜如玉, 沿着, 岁月, 的, 长河, 跋涉, ...",1
1,"[这是, 英超, 被, 黑, 的, 最惨, 的, 一次, [二哈], [二哈], 十几年来,...",0
2,"[中国, 远洋, 海运, 集团, 副总经理, 俞曾, 港, 月, 日, 在, 上, 表示, ...",1
3,"[看, 流星花园, 其实, 也, 还好, 啦, 现在, 的, 观念, 以及, 时尚, 眼光,...",1
4,"[汉武帝, 的, 罪己, 诏, 的, 真实性, 尽管, 存在, 着, 争议, 然而, 轮台,...",1


加载停用词

In [4]:
stopwords = []
with open("stopwords.txt", "r", encoding="utf8") as f:
    for w in f:
        stopwords.append(w.strip())

TfIdf

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
data_str = [" ".join(content) for content, sentiment in train_data] + \
            [" ".join(content) for content, sentiment in test_data]
tfidf = TfidfVectorizer(token_pattern='\[?\w+\]?', stop_words=stopwords)
tfidf_fit = tfidf.fit_transform(data_str)

加载之前训练好的FastText模型

In [6]:
from gensim.models import FastText
model = FastText.load("model/model_100.txt")

最多只保留Tf-Idf最高的前多少个词

In [7]:
key_words = 20

根据Tfidf和FastText词向量计算句向量

In [8]:
X_train, y_train = [], []
for content, sentiment in train_data:
    X, y = [], sentiment
    X_tfidf = tfidf.transform([" ".join(content)]).toarray()
    keywords_index = np.argsort(-X_tfidf)[0, :key_words]
    for w in content:
        if w in model and w in tfidf.vocabulary_ and tfidf.vocabulary_[w] in keywords_index:
            X.append(np.expand_dims(model[w], 0) * X_tfidf[0, tfidf.vocabulary_[w]])
    if X:
        X = np.concatenate(X)
        X = np.expand_dims(np.mean(X, axis=0), 0)
        X_train.append(X)
        y_train.append(y)

/home/albert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/home/albert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [9]:
X_test, y_test = [], []
for content, sentiment in test_data:
    X, y = [], sentiment
    X_tfidf = tfidf.transform([" ".join(content)]).toarray()
    keywords_index = np.argsort(-X_tfidf)[0, :key_words]
    for w in content:
        if w in model and w in tfidf.vocabulary_ and tfidf.vocabulary_[w] in keywords_index:
            X.append(np.expand_dims(model[w], 0) * X_tfidf[0, tfidf.vocabulary_[w]])
    if X:
        X = np.concatenate(X)
        X = np.expand_dims(np.mean(X, axis=0), 0)
        X_test.append(X)
        y_test.append(y)

/home/albert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/home/albert/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


### 神经网络

In [15]:
import tensorflow as tf
batch_size = 1000
lr = 1e-3
X = tf.placeholder(shape=(batch_size, 100), dtype=tf.float32, name="X")
y = tf.placeholder(shape=(batch_size, 1), dtype=np.float32, name="y")
with tf.variable_scope("fcn", reuse=tf.AUTO_REUSE):
    W1 = tf.get_variable("W1", shape=(100, 50))
    b1 = tf.get_variable("b1", shape=(50,))
    W2 = tf.get_variable("W2", shape=(50, 1))
    b2 = tf.get_variable("b2", shape=(1,))
    fcn1 = tf.nn.xw_plus_b(X, W1, b1)
    logists = tf.nn.xw_plus_b(fcn1, W2, b2)
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logists, labels=y))
    op = tf.train.AdamOptimizer(lr).minimize(loss)

In [11]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
config = tf.ConfigProto(gpu_options=gpu_options)
sess = tf.Session(config=config)

In [16]:
total_step = 10000
step = 0
cursor = 0
sess.run(tf.global_variables_initializer())
while step < total_step:
    _X, _y = X_train[cursor: cursor + batch_size], y_train[cursor: cursor + batch_size]
    cursor += batch_size
    if len(_X) < batch_size:
        cursor = batch_size - len(_X)
        _X += X_train[: cursor]
        _y += y_train[: cursor]
    _X = np.concatenate(_X)
    _y = np.reshape(np.array(_y, dtype=np.float32), (batch_size, 1))
    _, l = sess.run([op, loss], feed_dict={X: _X, y: _y})
    if step % 1000 == 0:
        print("step:", step, " loss:", l)
    step += 1

step: 0  loss: 0.69312716
step: 1000  loss: 0.46733403
step: 2000  loss: 0.52548593
step: 3000  loss: 0.5076738
step: 4000  loss: 0.527149
step: 5000  loss: 0.49963742
step: 6000  loss: 0.43283564
step: 7000  loss: 0.5325002
step: 8000  loss: 0.561318
step: 9000  loss: 0.57767063


In [17]:
_X = np.concatenate(X_test + [np.zeros_like(X_test[0])] * (batch_size - len(X_test)))

In [18]:
result = sess.run(tf.nn.sigmoid(logists), feed_dict={X: _X})
prediction = []
for i in result[:len(X_test)]:
    if i > 0.5:
        prediction.append(1)
    else:
        prediction.append(0)

In [19]:
from sklearn import metrics
print(metrics.classification_report(y_test, prediction))
print("准确率:", metrics.accuracy_score(y_test, prediction))

             precision    recall  f1-score   support

          0       0.67      0.74      0.70       155
          1       0.88      0.83      0.86       344

avg / total       0.81      0.81      0.81       499

准确率: 0.8056112224448898
